순위
제목
코드
내용
줄거리
평점
댓글

In [1]:
import requests
from bs4 import BeautifulSoup
import MySQLdb
from urllib import parse

In [2]:
db = MySQLdb.connect(host="localhost",  # host = localhost # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1234",     # password  # 1234
                     db="naver_movie")  # DB NAME   # 자신이 만든 데이터베이스 이름 
# connect() 함수: 인자가 4개 들어갑니다.

db.set_character_set('utf8')

In [3]:
db

<_mysql.connection open to 'localhost' at 7193438>

In [4]:
# 데이터베이스 커서(Cursor)는 일련의 데이터에 순차적으로 액세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소이다.
cur = db.cursor()
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=161967
cur

In [5]:
# 데이터 베이스 설계
# 우리는 영화정보, 댓글의 관계를 먼저 파악해야합니다. 
# 영화가 한개가 댓글을 여러개 가질 수 있습니다.
# 그러나 댓글 하나가 영화 여러개를 가질 수 없습니다. 
# 따라서 1 대 N 관계 
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(255) NOT NULL,
             story   TEXT,
             open_date  DATE)
             ENGINE INNODB;
        """ # ENGINE INNODB: 데이터베이스 타입
cur.execute(query)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: Warning: (1050, "Table 'naver_movie' already exists")
  from ipykernel import kernelapp as app


0

In [6]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            score    INTEGER    NOT NULL,
            conetent TEXT       NOT NULL,
            movie_code INTEGER,
            writer  VARCHAR(100),
            write_date DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (writer, write_date),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
            ;
"""
cur.execute(query)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: Warning: (1050, "Table 'movie_comment' already exists")
  from ipykernel import kernelapp as app


0

# Naver_Movie_Crawling
### 실습

In [ ]:
BASE_URL = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
params = {
    'sdb': 'sdb',
    'rank': 'rank',
    'rmovie.nhn': 'rmovie.nhn'
}

resp = requests.get(BASE_URL, params)
# resp
# resp.url
# resp.content
soup = BeautifulSoup(resp.content, 'html.parser')
# soup

In [ ]:
ranking = soup.find('tbody').find_all('tr')
ranking

In [ ]:
movie_list = []

for tr in ranking:
    movie_dict = {}
    
    try:
        a_tag = tr.find('td', class_="title").find('a')
        # print(title)
        link = a_tag['href']
        title = a_tag['title']
        code = link[28:]
        
        movie_dict['title'] = title
        movie_dict['link'] = link
        movie_dict['code'] = code
        movie_list.append(movie_dict)

    
    except AttributeError:
        pass
    
    
    
    
print(movie_list)

# Naver_Movie_해답

In [7]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

In [8]:
def get_movie_list(start_page=1, end_page=1):
    URL = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn"
    params = {
        'sel':'pnt',
        'date':20190616
    }
    
    movie_list = []
    for i in range(start_page, end_page+1):
        params['page'] = i
        
        resp = requests.get(URL, params)
        soup = BeautifulSoup(resp.content)
        
        list_ranking = soup.find(class_='list_ranking').find_all('tr')
        for movie_tr in list_ranking:
            try:
                code = movie_tr.find('a')['href'].split('=')[-1]
                title = movie_tr.find('a').text
            
                movie_list.append({
                    'code':code,
                    'title':title
                })
            except Exception:
                pass
            
    return movie_list

In [9]:
get_movie_list()

[{'code': '171539', 'title': '그린 북'},
 {'code': '174830', 'title': '가버나움'},
 {'code': '144906', 'title': '베일리 어게인'},
 {'code': '163788', 'title': '알라딘'},
 {'code': '169240', 'title': '아일라'},
 {'code': '151196', 'title': '원더'},
 {'code': '157243', 'title': '당갈'},
 {'code': '156464', 'title': '보헤미안 랩소디'},
 {'code': '17421', 'title': '쇼생크 탈출'},
 {'code': '154667', 'title': '덕구'},
 {'code': '10200', 'title': '터미네이터 2'},
 {'code': '69105', 'title': '월-E'},
 {'code': '10016', 'title': '나 홀로 집에'},
 {'code': '18988', 'title': '라이언 일병 구하기'},
 {'code': '22126', 'title': '인생은 아름다워'},
 {'code': '24452', 'title': '매트릭스'},
 {'code': '10102', 'title': '사운드 오브 뮤직'},
 {'code': '136900', 'title': '어벤져스: 엔드게임'},
 {'code': '10002', 'title': '빽 투 더 퓨쳐'},
 {'code': '17159', 'title': '포레스트 검프'},
 {'code': '82432', 'title': '헬프'},
 {'code': '92125', 'title': '헌터 킬러'},
 {'code': '29217', 'title': '글래디에이터'},
 {'code': '35901', 'title': '살인의 추억'},
 {'code': '66463', 'title': '토이 스토리 3'},
 {'code': '32686', 'titl

In [10]:
def get_movie_info(movie):
    code = movie.get('code')
    title = movie.get('title')
    
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code': code,
    }
    
    resp = requests.get(base_url, params)
    soup = BeautifulSoup(resp.content)
    
    open_date_tag = soup.find('dl', class_='info_spec').find('dd').find_all('span')[-1]

    open_date = ""
    for date in open_date_tag.find_all('a'):
        open_date += date.text
    
    open_date = "-".join(open_date.strip().split('.'))
    
    story = soup.find('div', class_='story_area')
    
    movie['story'] = story
    movie['open_date'] = open_date
    
    return movie


In [11]:
movie_list = get_movie_list()
movie_list[0]

{'code': '171539', 'title': '그린 북'}

In [12]:
get_movie_info(movie_list[0])

{'code': '171539', 'title': '그린 북', 'story': <div class="story_area">
 <div class="title_area">
 <h4 class="h_story"><strong class="blind">줄거리</strong></h4>
 </div>
 <h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
 원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
 취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
 <p class="con_tx">1962년 미국, 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인
 <br/> 천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다.
 <br/> 
 <br/> 백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는
 <br/> 위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고,
 <br/> 투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다.
 <br/> 
 <br/> 거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사.
 <br/> 생각, 행동, 말투, 취향까지 달라도 너무 다른 두 사람은
 <br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
 <button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
 </div>, 'open_date': '2019-01-09'}

In [20]:
# query = """
#         CREATE TABLE IF NOT EXISTS naver_movie
#             (code    INTEGER      NOT NULL      PRIMARY KEY,
#              created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
#              title   VARCHAR(100) NOT NULL,
#              story   TEXT,
#              open_date  DATE)
#         """

def insert_movie(movie):
    code = movie.get('code')
    title = movie.get('title')
    story = movie.get('story')
    
    story = str(story)
    
    open_date = movie.get('open_date')
    
    story = db.escape_string(story)
    
    query = """
    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('{0}', '{1}', %s, '{2}')
    """.format(code, title, open_date)
    print(query)
    cur.execute(query, [story])
    db.commit()

In [21]:
sample_movie = get_movie_info(movie_list[0])
sample_movie

{'code': '171539', 'title': '그린 북', 'story': <div class="story_area">
 <div class="title_area">
 <h4 class="h_story"><strong class="blind">줄거리</strong></h4>
 </div>
 <h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
 원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
 취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
 <p class="con_tx">1962년 미국, 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인
 <br/> 천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다.
 <br/> 
 <br/> 백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는
 <br/> 위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고,
 <br/> 투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다.
 <br/> 
 <br/> 거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사.
 <br/> 생각, 행동, 말투, 취향까지 달라도 너무 다른 두 사람은
 <br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
 <button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
 </div>, 'open_date': '2019-01-09'}

In [22]:
insert_movie(sample_movie)


    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('171539', '그린 북', %s, '2019-01-09')
    


IntegrityError: (1062, "Duplicate entry '171539' for key 'PRIMARY'")

In [24]:
from MySQLdb import IntegrityError

movie_list = get_movie_list()
for movie in movie_list:
    
    movie = get_movie_info(movie)
    try:
        insert_movie(movie)
    except IntegrityError :
        pass
    except Exception:
        pass
    
    


    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('171539', '그린 북', %s, '2019-01-09')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('174830', '가버나움', %s, '2019-01-24')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('144906', '베일리 어게인', %s, '2018-11-22')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('163788', '알라딘', %s, '2019-05-23')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('169240', '아일라', %s, '2018-06-21')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('151196', '원더', %s, '2017-12-27')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('157243', '당갈', %s, '2018-04-25')
    

    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('156464', '보헤미안 랩소디', %s, '2018-10-31')
    

    INSERT INTO naver_movie
    (code, title, story, open_date

In [52]:
def get_comment_list(code, last_page=1):
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn'
    params = {
        'code': code,
        'type': 'after',
        'order': 'newest'
    }
    result_list = []
    for page in range(1, last_page+1):
        params['page'] = page
        
        resp = requests.get(url, params)
        soup = BeautifulSoup(resp.content)
        reple_list = soup.find('div', class_='score_result').find_all('li')
        
        for reple in reple_list:
            score = reple.find(class_='star_score').text.strip()
            content = reple.find('p')
            name_n_date = content.find_next_sibling('dl').find_all('em')
            
            content = content.text
            
            nickname = name_n_date[0]
            date = name_n_date[1]
            result_list.append({
                'write_name':nickname,
                'write_date': date.text,
                'content': content,
                'score': score,
                'code': code
            })
    return result_list
        

In [53]:
get_comment_list('156464')

[{'write_name': <em>
  <a href="#" onclick="javascript:showPointListByNid(15815091, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
  <span>신영(youn****)</span>
  </a>
  </em>,
  'write_date': '2019.06.16 18:48',
  'content': '관람객난 이미 퀸을 알고 있었다  ',
  'score': '10',
  'code': '156464'},
 {'write_name': <em>
  <a href="#" onclick="javascript:showPointListByNid(15810155, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
  <span>태바기(nogo****)</span>
  </a>
  </em>,
  'write_date': '2019.06.15 00:02',
  'content': '진짜 공감이다 이거!!!!  ',
  'score': '10',
  'code': '156464'},
 {'write_name': <em>
  <a href="#" onclick="javascript:showPointListByNid(15806789, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
  <span>소(aksg****)</span>
  </a>
  </em>,
  'write_date': '2019.06.13 23:50',
  'content': '관람객와 보다가 울었음..너무 슬펐음 ..진짜루다  ',
  'score': '10',
  'code': '156464'},
 {'write_name': <e

In [58]:
def insert_comment(comment):
    score = comment.get('score')
    writer = comment.get('writer')
    write_date = comment.get('write_date')
    write_date = '-'.join(write_date.split('.'))
    
    content = comment.get('content')
    movie_code = comment.get('code')
    

    
    query = """
    INSERT INTO movie_comment(score, writer, write_date, conetent, movie_code)
    VALUES('{}', '{}', '{}', '{}', '{}')
    """.format(score, writer, write_date, content, movie_code)
    
    cur.execute(query)
    db.commit()

In [59]:
comment_list = get_comment_list('171539')
for comment in comment_list:
    insert_comment(comment)

In [61]:
for movie in movie_list:
    try:
        comment_list = get_comment_list(movie['code'])
        for comment in comment_list:
            insert_comment(comment)
    except Exception:
        pass